In [39]:
!pip install tensorflow


In [40]:
!pip install opencv-python


In [41]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D,Activation,LeakyReLU,BatchNormalization,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [42]:
import pickle
import cv2
import os
from os import listdir
from sklearn.preprocessing import LabelBinarizer,MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [43]:
root_dir =  r'PlantVillage'

os.chdir(root_dir)
listdir()

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'PlantVillage'

In [ ]:
# Dimension of resized image
DEFAULT_IMAGE_SIZE = tuple((256, 256))

# Number of images used to train the model
N_IMAGES = 100

data_dir = '.'

"""We use the function `convert_image_to_array` to resize an image
to the size `DEFAULT_IMAGE_SIZE` we defined above."""

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

        return None

In [ ]:
print("Load images from all classes ...")
plant_disease_folder_list = listdir(data_dir)
print(len(plant_disease_folder_list))

# Loading Training & Validation Data

In [ ]:
image_list = []
label_list = []

print("Loading images...")

for class_name in os.listdir('.'):
    if not os.path.isdir(class_name):
        continue

    print("Processing:", class_name)

    for img in os.listdir(class_name):
        img_path = os.path.join(class_name, img)

        try:
            img_arr = cv2.imread(img_path)
            if img_arr is None:
                continue

            img_arr = cv2.resize(img_arr, (256, 256))
            image_list.append(img_arr)
            label_list.append(class_name)

        except:
            continue

print("Done loading images")


In [ ]:
print(len(image_list))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Transform the loaded training image data into numpy array
np_image_list = np.array(image_list, dtype=np.float16) / 255.0

# Check the number of images loaded for training
image_len = len(image_list)
print(f"Total number of images: {image_len}")

In [ ]:
# --- FIX: make sure labels are valid and flat ---
import numpy as np

print("Images loaded:", len(image_list))
print("Labels loaded:", len(label_list))

# remove any None or empty labels (safety)
label_list = [lbl for lbl in label_list if lbl is not None]

# convert to numpy array (required by LabelBinarizer)
label_list = np.array(label_list)

print("Clean labels:", label_list[:10])


# One-Hot Encoding the labels

In [ ]:
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list)

n_classes = len(label_binarizer.classes_)
print("Total number of classes:", n_classes)


In [ ]:
"""# Augment and Split Dataset
Using `ImageDataGenerator` to augment data by performing various operations on the training images.
"""

augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

In [ ]:
"""Splitting the data into training and test sets for validation purpose."""

x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)
print('Successfully split data into TRAIN & TEST')

In [ ]:
"""# Build Model
Defining the hyperparameters of the plant disease classification model.
"""

EPOCHS = 10
STEPS = 100
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
DEPTH = 3


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D,
    Flatten, Dense, Dropout, Activation
)
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LeakyReLU, BatchNormalization



In [ ]:
"""Creating a Sequential Model to build CNN for multi-class classification"""HEIGHT = 256
WIDTH = 256
DEPTH = 3

model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1

if K.image_data_format() == "channels_first":
    inputShape = (DEPTH, HEIGHT, WIDTH)
    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(1024, name = 'my_dense'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(n_classes))
model.add(Activation("softmax"))

model.summary()

## Feature Extraction from the Dense Layer

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input



In [ ]:
layer_name = "my_dense"

intermediate_layer_model = Model(
    inputs=model.inputs,
    outputs=model.get_layer(layer_name).output
)

intermediate_layer_model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
# Train Model
LR = 0.001   

opt = Adam(learning_rate=LR)

model.compile(
    loss="categorical_crossentropy",
    optimizer=opt,
    metrics=["accuracy"]
)

print("Training CNN...")

history = model.fit(
    augment.flow(x_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(x_test, y_test),
    epochs=20,
    verbose=1
)

scores = model.evaluate(x_test, y_test, verbose=0)
print(f"Test Accuracy: {scores[1] * 100:.2f}%")


In [ ]:
"""# Evaluate Model
Comparing the accuracy and loss by plotting the graph for training and validation.
"""

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

# Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()

# Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

"""Evaluating model accuracy by using the `evaluate` method"""

print("[INFO] Calculating model accuracy")
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
#Saving the Model

# model.save('PDD_completemodel')
# model.save('PDD_completemodel.h5')
# intermediate_layer_model.save('PDD_IntermediateModel')
# intermediate_layer_model.save('PDD_IntermediateModel.h5')

In [ ]:
x_train_predict = intermediate_layer_model.predict(x_train)
# print(x_train_predict.shape)

x_test_predict = intermediate_layer_model.predict(x_test)
print(x_test_predict.shape)

# Support Vector Machine

Integration of CNN with Support Vector Machine.

In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')

svm.fit(x_train_predict,np.argmax(y_train,axis=1))

print('SVM Fit Complete')

In [ ]:
svm.score(x_train_predict,np.argmax(y_train,axis=1))

In [ ]:
svm.score(x_test_predict,np.argmax(y_test,axis=1))

#Save the SVM model in pickle file
# pickle.dump(svm,open('svms.pkl', 'wb'))

In [ ]:
Pred_labels = svm.predict(x_test_predict)
Pred_labels = pd.DataFrame(Pred_labels,index =None)
Pred_labels.head()

# XGBoost Integration

Integrating the CNN model with Extreme Gradient Boost algorithm.

In [ ]:
!pip install xgboost

In [ ]:
import xgboost as xgb

xb = xgb.XGBClassifier(use_label_encoder=False)

xb.fit(x_train_predict,np.argmax(y_train,axis=1))

print('XGBoost Fit Complete')

#Save XGBoost Model
# pickle.dump(xb,open('xgb_model.pkl','wb'))

In [ ]:
xb.score(x_train_predict,np.argmax(y_train,axis=1))

In [ ]:
xb.score(x_test_predict,np.argmax(y_test,axis=1))

In [ ]:
# from tensorflow.keras.models import load_model

# ai = load_model('D:\PDD_Models\HDF5\PDD_Intermediatemodel.h5')
# xgb_model = xgb.XGBClassifier()
# xgb_model.load_model(r'D:\PDD_Models\xgb_model.json')

# x2 = ai.predict(x_test)
# xgb_model.score(x_test_predict,np.argmax(y_test,axis=1))

In [ ]:
# ================== SAVE MODELS ==================

# Save CNN model
model.save("plant_disease_cnn.h5")

# Save XGBoost model
import pickle
pickle.dump(xb, open("plant_disease_xgb.pkl", "wb"))

print("Models saved successfully")


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load CNN model
model = load_model("plant_disease_cnn.h5")
print("CNN model loaded")

CLASS_NAMES = [
    "Apple___Apple_scab",
    "Apple___Black_rot",
    "Apple___Cedar_apple_rust",
    "Cherry_(including_sour)___Powdery_mildew",
    "Grape___Black_rot",
    "Peach___Bacterial_spot",
    "Strawberry___Leaf_scorch",
    "Tomato___Leaf_Mold"
]

IMG_SIZE = 256

def predict_disease(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found")

    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img)
    idx = np.argmax(preds)

    return CLASS_NAMES[idx], preds[0][idx] * 100

# TEST
disease, confidence = predict_disease(
    r"C:\Users\Lenovo\InstallConda\Detection-of-Plant-Disease-main\test_images\rotten.jpg"
)

print("🦠 Disease:", disease)
print("🎯 Confidence:", round(confidence, 2), "%")


In [ ]:
# Save CNN model
model.save("plant_disease_cnn.h5")

# Save Label Binarizer
import pickle
pickle.dump(label_binarizer, open("plant_disease_label_transform.pkl", "wb"))

print("Models saved successfully")
